In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

def read_data_function(file_name):
    data_frame = pd.read_csv(file_name, sep = ",")
    data_frame = data_frame[["date", "open"]]
    data_frame["date"] = pd.to_datetime(data_frame["date"], format = "%Y-%m-%d")
    data_frame.index = data_frame.pop("date")
    #scaler = MinMaxScaler(feature_range = (-1, 1))
    #stock_price_val = data_frame["open"].values.reshape(-1,1)
    #data_frame["open"] = scaler.fit_transform(stock_price_val)
    
    
    return data_frame

In [2]:
read_data_function("all_stocks_5yr.csv")

,open
date,
2013-02-08,15.07
2013-02-11,14.89
2013-02-12,14.45
2013-02-13,14.30
2013-02-14,14.94
...,...
2018-02-01,76.84
2018-02-02,77.53
2018-02-05,76.64


In [3]:
import numpy as np
import pandas as pd
def data_prep_function(sequence_dataset,window_size):
    #sequence_dataset = [np.array(sequence_dataset[i*sliding_window_size: (i + 1) * sliding_window_size]) for i in range(len(sequence_dataset) // sliding_window_size)]
    #X = np.array([sequenc_dataset[i: i + num_steps] for i in range(len(sequence_dataset) - sliding_window_size)])
    #y = np.array([sequence_dataset[i + num_steps] for i in range(len(sequence_dataset) - sliding_window_size)])
    for i in range(1, window_size + 1):
        sequence_dataset[f"open-{i}"] = sequence_dataset["open"].shift(i)


    sequence_dataset.dropna(inplace = True)


    return sequence_dataset

In [4]:
data = read_data_function("all_stocks_5yr.csv")
window_data = data_prep_function(data, window_size = 10)

In [5]:
from sklearn.preprocessing import MinMaxScaler
def matrix_formatting_and_normalize_dataset(sliding_window_dataset):
    sliding_window_dataset = sliding_window_dataset.to_numpy()
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(sliding_window_dataset)
    return normalized_data

In [6]:
data = read_data_function("all_stocks_5yr.csv")
window_data = data_prep_function(data, window_size = 10)
matrix_formatting_and_normalize_dataset(window_data)

array([[0.00586571, 0.00585102, 0.0058755 , ..., 0.00628189, 0.00649732,
        0.00658545],
       [0.00564048, 0.00586571, 0.00585102, ..., 0.00620844, 0.00628189,
        0.00649732],
       [0.00570903, 0.00564048, 0.00586571, ..., 0.0065218 , 0.00620844,
        0.00628189],
       ...,
       [0.03673166, 0.03716742, 0.03682958, ..., 0.03747589, 0.03705481,
        0.03674634],
       [0.03482212, 0.03673166, 0.03716742, ..., 0.03762767, 0.03747589,
        0.03705481],
       [0.03480253, 0.03482212, 0.03673166, ..., 0.03812709, 0.03762767,
        0.03747589]])

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch
def feature_target_modeling(dataset) -> tuple[TensorDataset]:
    X = dataset[:, 1:]
    y = dataset[:,0]
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], 1))
    X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype = torch.float32)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor) 
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 16, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = False)
    return train_loader, test_loader

In [20]:
data = read_data_function("all_stocks_5yr.csv")
window_data = data_prep_function(data, window_size = 10)
df = matrix_formatting_and_normalize_dataset(window_data)
feature_target_modeling(df)

(<torch.utils.data.dataloader.DataLoader at 0x1540448f0>,
 <torch.utils.data.dataloader.DataLoader at 0x1584c66f0>)